**Stat 451 Final Project- Data Cleaning- One Hot Encoding**

*Dylan Asoh, Logan O'Brien, Samuel Peterson*

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
data = pd.read_csv("train.csv")

In [3]:
data_drop = data.drop(['host_has_profile_pic', 'host_identity_verified','thumbnail_url', 'first_review', 'last_review', 'host_response_rate', 'neighbourhood', 'description', 'latitude', 'longitude', 'host_since', 'id', 'name'], axis=1)
data_drop.columns

Index(['log_price', 'property_type', 'room_type', 'amenities', 'accommodates',
       'bathrooms', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'city',
       'instant_bookable', 'number_of_reviews', 'review_scores_rating',
       'zipcode', 'bedrooms', 'beds'],
      dtype='object')

In [4]:
data_drop = data_drop.drop(['review_scores_rating', 'zipcode'], axis=1)
data_drop.columns

Index(['log_price', 'property_type', 'room_type', 'amenities', 'accommodates',
       'bathrooms', 'bed_type', 'cancellation_policy', 'cleaning_fee', 'city',
       'instant_bookable', 'number_of_reviews', 'bedrooms', 'beds'],
      dtype='object')

In [5]:
data_drop.isna().sum()

log_price                0
property_type            0
room_type                0
amenities                0
accommodates             0
bathrooms              200
bed_type                 0
cancellation_policy      0
cleaning_fee             0
city                     0
instant_bookable         0
number_of_reviews        0
bedrooms                91
beds                   131
dtype: int64

In [6]:
data_drop = data_drop.drop(11632)

In [7]:
data_clean_ohe = data_drop.copy()

In [8]:
row2=[]
final = []
amentity_totals_dict = {}
for row in data_clean_ohe['amenities']:
    row2 = (row.split(","))
    for item in row2:
        amenity = re.findall("[A-Za-z\s]+", item)
        if len(amenity) == 0:
            pass
        elif amenity[0] not in final:
            final.append(amenity[0])
            amentity_totals_dict[amenity[0]] = 1
        else:
            amentity_totals_dict[amenity[0]] += 1
def amentity_totals(listing):
    if len(listing) != 0:
        amenities = (listing.split(","))
        if "translation missing: en.hosting_amenity_50" in amenities:
            return len(amenities - 1)
        else:
            return len(amenities)
    else:
        return 0

data_clean_ohe["amenities"] = data_clean_ohe["amenities"].apply(amentity_totals)

In [9]:
def bed_type(bed):
    if bed == 'Real Bed':
        return 4
    elif bed == 'Pull-out Sofa':
        return 3
    elif bed == 'Futon':
        return 2
    elif bed == 'Couch':
        return 1
    else: #Airbed
        return 0

In [10]:
bed_type_dum = pd.get_dummies(data_drop["bed_type"])
data_clean_ohe = pd.concat([data_clean_ohe, bed_type_dum], axis=1)
data_clean_ohe = data_clean_ohe.drop(["bed_type"], axis=1)
data_clean_ohe

,log_price,property_type,room_type,amenities,accommodates,bathrooms,cancellation_policy,cleaning_fee,city,instant_bookable,number_of_reviews,bedrooms,beds,Airbed,Couch,Futon,Pull-out Sofa,Real Bed
0,5.010635,Apartment,Entire home/apt,9,3,1.0,strict,True,NYC,f,2,1.0,1.0,0,0,0,0,1
1,5.129899,Apartment,Entire home/apt,15,7,1.0,strict,True,NYC,t,6,3.0,3.0,0,0,0,0,1
2,4.976734,Apartment,Entire home/apt,19,5,1.0,moderate,True,NYC,t,10,1.0,3.0,0,0,0,0,1
3,6.620073,House,Entire home/apt,15,4,1.0,flexible,True,SF,f,0,2.0,2.0,0,0,0,0,1
4,4.744932,Apartment,Entire home/apt,12,2,1.0,moderate,True,DC,t,4,0.0,1.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74106,4.605170,Apartment,Private room,1,1,1.0,flexible,False,NYC,f,0,1.0,1.0,0,0,0,0,1
74107,5.043425,Apartment,Entire home/apt,16,4,2.0,moderate,True,LA,f,16,2.0,4.0,0,0,0,0,1
74108,5.220356,Apartment,Entire home/apt,31,5,1.0,moderate,True,NYC,t,43,2.0,2.0,0,0,0,0,1
74109,5.273000,Apartment,Entire home/apt,15,2,1.0,strict,True,NYC,t,0,0.0,2.0,0,0,0,0,1


In [11]:
def cancellation_policy(policy):
    if policy == "flexible":
        return 3
    elif policy == "moderate":
        return 2
    elif policy =="strict":
        return 1
    else: # there are weird ambiguous "super" strict values that have _60 and _30 at end
        return 0

In [12]:
can_pol_dum = pd.get_dummies(data_drop["cancellation_policy"])
data_clean_ohe = pd.concat([data_clean_ohe, can_pol_dum], axis=1)
data_clean_ohe = data_clean_ohe.drop(["cancellation_policy"], axis=1)
data_clean_ohe

,log_price,property_type,room_type,amenities,accommodates,bathrooms,cleaning_fee,city,instant_bookable,number_of_reviews,...,Airbed,Couch,Futon,Pull-out Sofa,Real Bed,flexible,moderate,strict,super_strict_30,super_strict_60
0,5.010635,Apartment,Entire home/apt,9,3,1.0,True,NYC,f,2,...,0,0,0,0,1,0,0,1,0,0
1,5.129899,Apartment,Entire home/apt,15,7,1.0,True,NYC,t,6,...,0,0,0,0,1,0,0,1,0,0
2,4.976734,Apartment,Entire home/apt,19,5,1.0,True,NYC,t,10,...,0,0,0,0,1,0,1,0,0,0
3,6.620073,House,Entire home/apt,15,4,1.0,True,SF,f,0,...,0,0,0,0,1,1,0,0,0,0
4,4.744932,Apartment,Entire home/apt,12,2,1.0,True,DC,t,4,...,0,0,0,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74106,4.605170,Apartment,Private room,1,1,1.0,False,NYC,f,0,...,0,0,0,0,1,1,0,0,0,0
74107,5.043425,Apartment,Entire home/apt,16,4,2.0,True,LA,f,16,...,0,0,0,0,1,0,1,0,0,0
74108,5.220356,Apartment,Entire home/apt,31,5,1.0,True,NYC,t,43,...,0,0,0,0,1,0,1,0,0,0
74109,5.273000,Apartment,Entire home/apt,15,2,1.0,True,NYC,t,0,...,0,0,0,0,1,0,0,1,0,0


In [13]:
prop_dummies = pd.get_dummies(data_clean_ohe["property_type"])
prop_dummies

,Apartment,Bed & Breakfast,Boat,Boutique hotel,Bungalow,Cabin,Camper/RV,Casa particular,Castle,Cave,...,Serviced apartment,Tent,Timeshare,Tipi,Townhouse,Train,Treehouse,Vacation home,Villa,Yurt
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74106,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
74107,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
74108,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
74109,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
data_clean_ohe = pd.concat([data_clean_ohe, prop_dummies], axis=1)
data_clean_ohe = data_clean_ohe.drop(["property_type"], axis=1)
data_clean_ohe

,log_price,room_type,amenities,accommodates,bathrooms,cleaning_fee,city,instant_bookable,number_of_reviews,bedrooms,...,Serviced apartment,Tent,Timeshare,Tipi,Townhouse,Train,Treehouse,Vacation home,Villa,Yurt
0,5.010635,Entire home/apt,9,3,1.0,True,NYC,f,2,1.0,...,0,0,0,0,0,0,0,0,0,0
1,5.129899,Entire home/apt,15,7,1.0,True,NYC,t,6,3.0,...,0,0,0,0,0,0,0,0,0,0
2,4.976734,Entire home/apt,19,5,1.0,True,NYC,t,10,1.0,...,0,0,0,0,0,0,0,0,0,0
3,6.620073,Entire home/apt,15,4,1.0,True,SF,f,0,2.0,...,0,0,0,0,0,0,0,0,0,0
4,4.744932,Entire home/apt,12,2,1.0,True,DC,t,4,0.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74106,4.605170,Private room,1,1,1.0,False,NYC,f,0,1.0,...,0,0,0,0,0,0,0,0,0,0
74107,5.043425,Entire home/apt,16,4,2.0,True,LA,f,16,2.0,...,0,0,0,0,0,0,0,0,0,0
74108,5.220356,Entire home/apt,31,5,1.0,True,NYC,t,43,2.0,...,0,0,0,0,0,0,0,0,0,0
74109,5.273000,Entire home/apt,15,2,1.0,True,NYC,t,0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
def room_type(room):
    if room == "Entire home/apt":
        return 2
    elif room == "Private room":
        return 1
    else: #Shared Room
        return 0

In [16]:
room_type_dum = pd.get_dummies(data_drop["room_type"])
data_clean_ohe = pd.concat([data_clean_ohe, room_type_dum], axis=1)
data_clean_ohe = data_clean_ohe.drop(["room_type"], axis=1)
data_clean_ohe

,log_price,amenities,accommodates,bathrooms,cleaning_fee,city,instant_bookable,number_of_reviews,bedrooms,beds,...,Tipi,Townhouse,Train,Treehouse,Vacation home,Villa,Yurt,Entire home/apt,Private room,Shared room
0,5.010635,9,3,1.0,True,NYC,f,2,1.0,1.0,...,0,0,0,0,0,0,0,1,0,0
1,5.129899,15,7,1.0,True,NYC,t,6,3.0,3.0,...,0,0,0,0,0,0,0,1,0,0
2,4.976734,19,5,1.0,True,NYC,t,10,1.0,3.0,...,0,0,0,0,0,0,0,1,0,0
3,6.620073,15,4,1.0,True,SF,f,0,2.0,2.0,...,0,0,0,0,0,0,0,1,0,0
4,4.744932,12,2,1.0,True,DC,t,4,0.0,1.0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74106,4.605170,1,1,1.0,False,NYC,f,0,1.0,1.0,...,0,0,0,0,0,0,0,0,1,0
74107,5.043425,16,4,2.0,True,LA,f,16,2.0,4.0,...,0,0,0,0,0,0,0,1,0,0
74108,5.220356,31,5,1.0,True,NYC,t,43,2.0,2.0,...,0,0,0,0,0,0,0,1,0,0
74109,5.273000,15,2,1.0,True,NYC,t,0,0.0,2.0,...,0,0,0,0,0,0,0,1,0,0


In [17]:
city_dummies = pd.get_dummies(data_clean_ohe["city"])
city_dummies

,Boston,Chicago,DC,LA,NYC,SF
0,0,0,0,0,1,0
1,0,0,0,0,1,0
2,0,0,0,0,1,0
3,0,0,0,0,0,1
4,0,0,1,0,0,0
...,...,...,...,...,...,...
74106,0,0,0,0,1,0
74107,0,0,0,1,0,0
74108,0,0,0,0,1,0
74109,0,0,0,0,1,0


In [18]:
data_clean_ohe = pd.concat([data_clean_ohe, city_dummies], axis=1)

In [19]:
data_clean_ohe = data_clean_ohe.drop(["city"], axis=1)

In [20]:
data_clean_ohe['cleaning_fee'] = data_clean_ohe['cleaning_fee'].map(({True:1,False:0}))
# converting instant_bookable to 0 or 1
data_clean_ohe['instant_bookable'] = data_clean_ohe['instant_bookable'].map(({'t':1,'f':0}))

In [21]:
# y = data_clean_ohe["log_price"]
data_clean_ohe

,log_price,amenities,accommodates,bathrooms,cleaning_fee,instant_bookable,number_of_reviews,bedrooms,beds,Airbed,...,Yurt,Entire home/apt,Private room,Shared room,Boston,Chicago,DC,LA,NYC,SF
0,5.010635,9,3,1.0,1,0,2,1.0,1.0,0,...,0,1,0,0,0,0,0,0,1,0
1,5.129899,15,7,1.0,1,1,6,3.0,3.0,0,...,0,1,0,0,0,0,0,0,1,0
2,4.976734,19,5,1.0,1,1,10,1.0,3.0,0,...,0,1,0,0,0,0,0,0,1,0
3,6.620073,15,4,1.0,1,0,0,2.0,2.0,0,...,0,1,0,0,0,0,0,0,0,1
4,4.744932,12,2,1.0,1,1,4,0.0,1.0,0,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74106,4.605170,1,1,1.0,0,0,0,1.0,1.0,0,...,0,0,1,0,0,0,0,0,1,0
74107,5.043425,16,4,2.0,1,0,16,2.0,4.0,0,...,0,1,0,0,0,0,0,1,0,0
74108,5.220356,31,5,1.0,1,1,43,2.0,2.0,0,...,0,1,0,0,0,0,0,0,1,0
74109,5.273000,15,2,1.0,1,1,0,0.0,2.0,0,...,0,1,0,0,0,0,0,0,1,0


In [22]:
print(data_clean_ohe.isna().sum())

log_price         0
amenities         0
accommodates      0
bathrooms       200
cleaning_fee      0
               ... 
Chicago           0
DC                0
LA                0
NYC               0
SF                0
Length: 63, dtype: int64


In [23]:
data_clean_ohe["bathrooms"] = data_clean_ohe["bathrooms"].fillna(data_clean_ohe["bathrooms"].mode()[0])
data_clean_ohe["beds"] = data_clean_ohe["beds"].fillna(data_clean_ohe["beds"].mode()[0])
data_clean_ohe["bedrooms"] = data_clean_ohe["bedrooms"].fillna(data_clean_ohe["beds"].mode()[0])

In [24]:
data_clean_ohe.isna().sum()

log_price       0
amenities       0
accommodates    0
bathrooms       0
cleaning_fee    0
               ..
Chicago         0
DC              0
LA              0
NYC             0
SF              0
Length: 63, dtype: int64

In [25]:
data_clean_ohe.isna().sum().value_counts()

0    63
dtype: int64

In [26]:
y = data_clean_ohe["log_price"]
# data_clean_ohe = data_clean_ohe.drop(["log_price"], axis=1)

In [27]:
data_clean_ohe.columns

Index(['log_price', 'amenities', 'accommodates', 'bathrooms', 'cleaning_fee',
       'instant_bookable', 'number_of_reviews', 'bedrooms', 'beds', 'Airbed',
       'Couch', 'Futon', 'Pull-out Sofa', 'Real Bed', 'flexible', 'moderate',
       'strict', 'super_strict_30', 'super_strict_60', 'Apartment',
       'Bed & Breakfast', 'Boat', 'Boutique hotel', 'Bungalow', 'Cabin',
       'Camper/RV', 'Casa particular', 'Castle', 'Cave', 'Chalet',
       'Condominium', 'Dorm', 'Earth House', 'Guest suite', 'Guesthouse',
       'Hostel', 'House', 'Hut', 'In-law', 'Island', 'Lighthouse', 'Loft',
       'Other', 'Parking Space', 'Serviced apartment', 'Tent', 'Timeshare',
       'Tipi', 'Townhouse', 'Train', 'Treehouse', 'Vacation home', 'Villa',
       'Yurt', 'Entire home/apt', 'Private room', 'Shared room', 'Boston',
       'Chicago', 'DC', 'LA', 'NYC', 'SF'],
      dtype='object')

In [28]:
data_clean_ohe.head()

,log_price,amenities,accommodates,bathrooms,cleaning_fee,instant_bookable,number_of_reviews,bedrooms,beds,Airbed,...,Yurt,Entire home/apt,Private room,Shared room,Boston,Chicago,DC,LA,NYC,SF
0,5.010635,9,3,1.0,1,0,2,1.0,1.0,0,...,0,1,0,0,0,0,0,0,1,0
1,5.129899,15,7,1.0,1,1,6,3.0,3.0,0,...,0,1,0,0,0,0,0,0,1,0
2,4.976734,19,5,1.0,1,1,10,1.0,3.0,0,...,0,1,0,0,0,0,0,0,1,0
3,6.620073,15,4,1.0,1,0,0,2.0,2.0,0,...,0,1,0,0,0,0,0,0,0,1
4,4.744932,12,2,1.0,1,1,4,0.0,1.0,0,...,0,1,0,0,0,0,1,0,0,0


In [29]:
ohe = data_clean_ohe.to_csv('ohe.csv')